In [1]:
import tensorflow.keras as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
import tf2onnx
import numpy as np
import math

In [2]:
n_classes = 10
nb_classes = 10
input_shape = (28, 28) #Lenet5
#batch_size = 500 # Lenet
batch_size = 500 # Lenet
input_shape = (28, 28) #Lenet5

In [3]:
img_rows, img_cols = 64, 64
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
#x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32') 
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_test)


print((x_train.shape,y_train.shape))
#print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((50000, 32, 32, 3), (50000, 10))
((10000, 32, 32, 3), (10000, 10))


In [4]:
training_id = 10
model_short_name = 'resnet'
framework = 'keras'

In [9]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/resnet18/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

print(t_elapsed_0)

0.6717648506164551


In [8]:
size1 = os.path.getsize(path+ model_name+'.h5')
size1

89756056

In [10]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [16]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx11/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools11/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/errors11/'
restored_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/restored11/'

In [25]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    with tf.device('/cpu:0'): 
        k_predict = model.predict(x)
    t_elapsed_1 = time.time() - since_1
    # Export the model
    
    if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
        os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
    if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
        os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
        
    since_1 = time.time()
    
    since_onnx = time.time()
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                input_signature=None, opset=11, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path=onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(k_predict-ort_outs[0])
    rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err)
    
    
    np.testing.assert_array_equal(k_predict, ort_outs[0])
    ## Converting the coremltool
    since_1 = time.time()
    coreml_model = coremltools.convert(model)
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_5 = time.time() - since_1
    
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    
    size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:x})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
    rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])
    
    for j in range (len(abs_err)):
        for k in range(len(abs_err[j])): 
            data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [26]:
def _lets_convert(data, x, y, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_): # for cifar10 etc
    since = time.time()
    batches = 0
    for x_batch, y_batch in data.flow(x, y, batch_size=batch_size):
        to_onnx(batches, x_batch,data_writer_error, data_writer_error2, data_writer_run, batch_size, input_)
        batches += 1
        if batches >= len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
        if batches == 10:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [27]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name, input_):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])

    for batch_size in [1, 5,10,50,100,128, 150,200, 250,300,350, 400, 450, 500]:
        print("################ Batch size: ", batch_size)
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

        datagen.fit(x_test)
        _lets_convert(datagen,x_test, y_test, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_)
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [28]:
# run expriment resnet18
input_ = 'input_1'
run_experiment(model_short_name, model_name, input_)

################ Batch size:  1
converting for batch:  0

*********




AssertionError: 
Arrays are not equal

Mismatched elements: 10 / 10 (100%)
Max absolute difference: 1.7695129e-07
Max relative difference: 1.073789e-05
 x: array([[7.138716e-06, 4.579959e-06, 1.647933e-02, 4.962262e-06,
        1.268875e-02, 2.943818e-03, 2.821678e-03, 5.195573e-06,
        9.586660e-01, 6.378663e-03]], dtype=float32)
 y: array([[7.138670e-06, 4.579960e-06, 1.647915e-02, 4.962267e-06,
        1.268877e-02, 2.943821e-03, 2.821665e-03, 5.195554e-06,
        9.586661e-01, 6.378688e-03]], dtype=float32)

## convert VGG

In [13]:
model_short_name = 'vgg'
input_ = 'input_3'

In [14]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/vgg/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_vgg-cifar10_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

71945464

In [15]:
size1 = os.path.getsize(path+ model_name+'.h5')
size1

71945464

In [16]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    with tf.device('/cpu:0'): 
        k_predict = model.predict(x)
    t_elapsed_1 = time.time() - since_1
    # Export the model
    
    if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
        os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
    if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
        os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
        
    since_1 = time.time()
    
    since_onnx = time.time()
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                input_signature=None, opset=11, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path=onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(k_predict-ort_outs[0])
    rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err)
    

    
    ## Converting the coremltool
    since_1 = time.time()
    coreml_model = coremltools.convert(model)
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_5 = time.time() - since_1
    
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    #print(name_1)

    size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:x})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
    rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])
    
    for j in range (len(abs_err)):
        for k in range(len(abs_err[j])): 
            data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [17]:
def _lets_convert(data, x, y, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_): # for cifar10 etc
    since = time.time()
    batches = 0
    for x_batch, y_batch in data.flow(x, y, batch_size=batch_size):
        to_onnx(batches, x_batch,data_writer_error, data_writer_error2, data_writer_run, batch_size, input_)
        batches += 1
        if batches >= len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
        if batches == 10:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [18]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name, input_):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])
    
    #for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150,200, 250,300,350, 400, 450, 500]:
    for batch_size in [1, 5,10,50,100,128, 150,200, 250,300,350, 400, 450, 500]:
        print("################ Batch size: ", batch_size)
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

        datagen.fit(x_test)
        _lets_convert(datagen,x_test, y_test, data_writer_error, data_writer_error2, data_writer_run, batch_size, input_)
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [19]:
run_experiment(model_short_name, model_name, input_)

################ Batch size:  1
converting for batch:  0

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 141/141 [00:01<00:00, 108.26 ops/s]


converting for batch:  1

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 141/141 [00:01<00:00, 107.89 ops/s]


converting for batch:  2

*********




...

converting for batch:  8

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 141/141 [00:01<00:00, 106.89 ops/s]


converting for batch:  9

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 141/141 [00:02<00:00, 69.26 ops/s] 


Conversion complete in 4m 47s


## Lets convert from ONNX back to keras

In [1]:
from onnx2keras import onnx_to_keras
from onnx_tf.backend import prepare

In [16]:
onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
tf_rep = prepare(onnx_model)  # prepare tf representation
print(tf_rep.inputs)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/keras/resnet/tf_resnet18-cifar10_2021-10-29_10.onnx'

In [20]:
coreml_model = coremltools.convert(model)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 141/141 [00:01<00:00, 97.80 ops/s]


In [26]:
split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
name_1 = split_[0].replace('name: "', '')
name_1 = name_1.replace('"', '')
print(name_1)

input_2


In [25]:
print(split_[0].replace('name: "', ''))

input_2"
